In [56]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from functools import partial
from math import log, sqrt, exp, isclose
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm, t
import scipy
import numpy as np
from scipy.stats import norm
from datetime import datetime
import inspect
import math
import sys
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime
import inspect

In [36]:
current_date = datetime(2022, 3, 13)
expire_date = datetime(2022, 4, 15)
T = (expire_date - current_date).days / 365
S = 165
X = 165
implied_vol = 0.2
r = 0.0425
coupon = 0.0053
b = r - coupon

In [55]:
# Functions
def d1(S, X, T, implied_vol, b):
    return (np.log(S / X) + (b + implied_vol ** 2 / 2) * T) / (implied_vol * np.sqrt(T))

def d2(d1, T, implied_vol):
    return d1 - implied_vol * np.sqrt(T)

def gbsm(option_type, S, X, T, implied_vol, r, b):
    call = 1 if option_type == "Call" else -1
    d_1 = d1(S, X, T, implied_vol, b)
    d_2 = d2(d_1, T, implied_vol)

    return call * (S * np.exp((b - r) * T) * norm.cdf(call * d_1, 0, 1) - X * np.exp(-r * T) * norm.cdf(call * d_2, 0, 1))

# Calculate first order derivative
def first_order_der(func, x, delta):
    return (func(x + delta) - func(x - delta)) / (2 * delta)

# Calculate second order derivative
def second_order_der(func, x, delta):
    return (func(x + delta) + func(x - delta) - 2 * func(x)) / delta ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-3):
    arg_names = list(inspect.signature(func).parameters.keys())
    derivative_fs = {1: first_order_der, 2: second_order_der}

    def partial_derivative(*args, **kwargs):
        args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
        arg_val = args_dict.pop(arg_name)

        def partial_f(x):
            p_kwargs = {arg_name: x, **args_dict}
            return func(**p_kwargs)

        return derivative_fs[order](partial_f, arg_val, delta)

    return partial_derivative

# Numerical Greeks
option_types = ["Call", "Put"]
greek_names = ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho', 'Carry Rho']
greek_funcs = {
    'Delta': cal_partial_derivative(gbsm, 1, 'S'),
    'Gamma': cal_partial_derivative(gbsm, 2, 'S'),
    'Vega': cal_partial_derivative(gbsm, 1, 'implied_vol'),
    'Theta': cal_partial_derivative(gbsm, 1, 'T', delta=-1e-6),
    'Rho': cal_partial_derivative(gbsm, 1, 'r'),
    'Carry Rho': cal_partial_derivative(gbsm, 1, 'b')
}

greeks_values = {greek_name: [greek_func(option_type, S, X, T, implied_vol, r, b) for option_type in option_types] for greek_name, greek_func in greek_funcs.items()}

# Modify the Theta values to be negative
greeks_values['Theta'] = [-theta for theta in greeks_values['Theta']]

# Output Greeks values
for greek_name, values in greeks_values.items():
    print(f"{greek_name}:")
    for option_type, value in zip(option_types, values):
        print(f"  {option_type}: {value:.8f}")
    print()

Delta:
  Call: 0.70730118
  Put: -0.29198757

Gamma:
  Call: 0.02491394
  Put: 0.02491399

Vega:
  Call: 18.99532994
  Put: 18.99532994

Theta:
  Call: -21.18953672
  Put: -15.86198627

Rho:
  Call: -1.26221101
  Put: -0.35637978

Carry Rho:
  Call: 14.34071461
  Put: -5.92012487



In [37]:
def n_nodes(N):
    return (N + 2) * (N + 1) // 2

def node_index(i, j):
    return n_nodes(j - 1) + i

def binomial_tree_no_div(option_type, S0, X, T, implied_vol, r, N):
    is_call = 1 if option_type == "Call" else -1
    dt = T / N
    disc = np.exp(-r * dt)
    u = np.exp(implied_vol * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    C = np.empty(n_nodes(N), dtype=float)
            
    for i in np.arange(N, -1, -1):
        for j in range(i, -1, -1):
            S = S0 * u ** j * d ** (i - j)
            index = node_index(j, i)
            C[index] = max(0, (S - X) * is_call)
            if i < N:
                val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
                C[index] = max(C[index], val)
                
    return C[0]

def binomial_tree(option_type, S0, X, T, div_time, div, implied_vol, r, N):
    if div_time is None or div is None:
        return binomial_tree_no_div(option_type, S0, X, T, implied_vol, r, N)
  
    is_call = 1 if option_type == "Call" else -1
    dt = T / N
    disc = np.exp(-r * dt)
    
    #calculate u, d, and p
    u = np.exp(implied_vol * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    new_T = T - div_time * dt
    new_N = N - div_time

    C = np.empty(n_nodes(div_time), dtype=float)
    for i in range(div_time, -1, -1):
        for j in range(i, -1, -1):
            S = S0 * u ** j * d ** (i - j)
            val_exe = max(0, (S - X) * is_call)
            if i < div_time:
                val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
            else:
                val = binomial_tree(option_type, S - div, X, new_T, None, None, implied_vol, r, new_N)
            C[node_index(j, i)] = max(val_exe, val)
    
    return C[0]

In [40]:
N = 500

value_no_div_call = binomial_tree_no_div("Call", S, X, T, implied_vol, r, N)
value_no_div_put = binomial_tree_no_div("Put", S, X, T, implied_vol, r, N)
print("Binomial tree value without dividend for call: " + str(value_no_div_call))
print("Binomial tree value without dividend for put: " + str(value_no_div_put))
div_date = datetime(2022, 4, 11)
div = 0.88
div_time = int((div_date - current_date).days / (expire_date - current_date).days * N)

value_call = binomial_tree("Call", S, X, T, div_time, div, implied_vol, r, N)
value_put = binomial_tree("Put", S, X, T, div_time, div, implied_vol, r, N)
print("Binomial tree value with dividend for call: " + str(value_call))
print("Binomial tree value with dividend for put: " + str(value_put))

Binomial tree value without dividend for call: 4.272824729808369
Binomial tree value without dividend for put: 3.6861285348239714
Binomial tree value with dividend for call: 4.111651103081258
Binomial tree value with dividend for put: 4.104293188582539


In [45]:
import numpy as np
import pandas as pd
import scipy.optimize
from datetime import datetime
import matplotlib.pyplot as plt

def implied_vol_american(option_type, S0, X, T, div_time, div, r, N, market_price, x0=0.5):
    def equation(implied_vol):
        return binomial_tree(option_type, S0, X, T, div_time, div, implied_vol, r, N) - market_price
    return scipy.optimize.fsolve(equation, x0=x0, xtol=0.00001)[0]

def calculate_sim_values(portfolios, sim_prices, days_ahead=0):
    sim_values = pd.DataFrame(index=portfolios.index, columns=list(range(sim_prices.shape[0])))
    sim_prices = np.array(sim_prices)

    for i in portfolios.index:
        if portfolios["Type"][i] == "Stock":
            single_values = sim_prices
        else:
            option_type = portfolios["OptionType"][i]
            X = portfolios["Strike"][i]
            T = ((portfolios["ExpirationDate"][i] - current_date).days - days_ahead) / 365
            implied_vol = portfolios["ImpliedVol"][i]
            div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
            div = 1
            option_values = [binomial_tree(option_type, S, X, T, div_time, div, implied_vol, r, N) for S in sim_prices]
            single_values = np.array(option_values)

        sim_values.loc[i, :] = portfolios["Holding"][i] * single_values

    sim_values['Portfolio'] = portfolios['Portfolio']
    return sim_values.groupby('Portfolio').sum()

# Reading the data
portfolios = pd.read_csv('problem2.csv', parse_dates=['ExpirationDate'])
portfolios['CurrentValue'] = portfolios['CurrentPrice'] * portfolios['Holding']

# Calculate the implied volatility for all portfolios
implied_vols = []
for i in range(len(portfolios.index)):
    if portfolios["Type"][i] == "Stock":
        implied_vols.append(None)
    else:
        option_type = portfolios["OptionType"][i]
        X = portfolios["Strike"][i]
        T = (portfolios["ExpirationDate"][i] - current_date).days / 365
        div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
        market_price = portfolios["CurrentPrice"][i]
        implied_vol = implied_vol_american(option_type, S, X, T, div_time, div, r, N, market_price)
        implied_vols.append(implied_vol)

# Store the implied volatility in portfolios
portfolios["ImpliedVol"] = implied_vols

# Simulate the price in 120-220 range
sim_prices = np.linspace(120, 220, 20)

# Calculate the stock and option values
sim_values = calculate_sim_values(portfolios, sim_prices, 10)
idx = 0
for portfolio, dataframe in sim_values.groupby('Portfolio'):
    i, j = idx // 3, idx % 3
    ax = axes[i][j]
    ax.plot(sim_prices, dataframe.iloc[0, :].values)
    ax.set_title(portfolio)
    ax.set_xlabel('Underlying Price', fontsize=8)
    ax.set_ylabel('Portfolio Value', fontsize=8)
    idx += 1

In [47]:
def calculate_returns(data, method="discrete", date_col="Date"):
    assert date_col in data.columns

    price_vars = list(data.columns)
    price_vars.remove(date_col)
    price_data = data[price_vars]

    num_vars = len(price_vars)
    num_rows = len(price_data) - 1

    returns_data = np.zeros(shape=(num_rows, num_vars))
    returns_df = pd.DataFrame(returns_data)

    for i in range(num_rows):
        for j in range(num_vars):
            returns_df.iloc[i, j] = price_data.iloc[i + 1, j] / price_data.iloc[i, j]
            if method == "discrete":
                returns_df.iloc[i, j] -= 1
            elif method == "log":
                returns_df.iloc[i, j] = math.log(returns_df.iloc[i, j])
            else:
                sys.exit(1)

    date_series = data[date_col].drop(index=0)
    date_series.index -= 1

    output = pd.DataFrame({date_col: date_series})

    for i in range(num_vars):
        output[price_vars[i]] = returns_df.iloc[:, i]

    return output

In [48]:
S = 151.03
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1
all_prices = pd.read_csv("DailyPrices.csv")
all_returns = calculate_returns(all_prices, method="discrete", date_col="Date")
all_returns

,Date,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,2/15/2022,0.016127,0.023152,0.018542,0.008658,0.053291,0.007987,0.008319,0.015158,0.091812,...,0.012807,-0.004082,0.004592,0.052344,0.003600,-0.012275,0.033021,0.026240,0.028572,0.013237
1,2/16/2022,0.001121,-0.001389,-0.001167,0.010159,0.001041,0.008268,0.007784,-0.020181,0.000604,...,0.006757,-0.002429,0.005763,0.038879,0.009294,0.012244,0.003363,0.015301,-0.001389,-0.025984
2,2/17/2022,-0.021361,-0.021269,-0.029282,-0.021809,-0.050943,-0.037746,-0.037669,-0.040778,-0.075591,...,-0.034949,0.005326,0.015017,-0.046988,-0.009855,0.004833,-0.030857,-0.031925,-0.033380,-0.028763
3,2/18/2022,-0.006475,-0.009356,-0.009631,-0.013262,-0.022103,-0.016116,-0.013914,-0.007462,-0.035296,...,-0.000646,-0.000908,0.007203,-0.000436,-0.003916,-0.005942,-0.013674,-0.004506,-0.003677,0.015038
4,2/22/2022,-0.010732,-0.017812,-0.000729,-0.015753,-0.041366,-0.004521,-0.008163,-0.019790,-0.010659,...,0.009494,0.007121,-0.008891,0.003243,-0.001147,-0.000673,0.008342,-0.037654,-0.002246,-0.013605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2/3/2023,-0.010629,0.024400,-0.023621,-0.084315,0.009083,-0.027474,-0.032904,-0.011866,-0.028053,...,-0.004694,-0.011251,-0.001277,-0.002677,0.038211,0.004134,0.002336,-0.008916,-0.005954,0.001617
244,2/6/2023,-0.006111,-0.017929,-0.006116,-0.011703,0.025161,-0.017942,-0.016632,-0.002520,-0.000521,...,-0.014451,0.003945,0.001066,-0.007102,0.022012,0.021826,-0.041181,0.005106,-0.009782,-0.004595
245,2/7/2023,0.013079,0.019245,0.042022,-0.000685,0.010526,0.046064,0.044167,0.029883,0.051401,...,-0.000368,-0.016473,-0.008518,0.019544,-0.003590,-0.001641,0.003573,0.001451,0.008669,-0.003618
246,2/8/2023,-0.010935,-0.017653,-0.003102,-0.020174,0.022763,-0.076830,-0.074417,-0.042741,0.001443,...,-0.008469,-0.004456,-0.001289,-0.018009,-0.004416,0.002819,-0.015526,0.004106,-0.015391,0.009363


In [51]:
# Simulate the prices based on returns with normal distribution
std = all_returns['AAPL'].std()
np.random.seed(0)
sim_returns = scipy.stats.norm(0, std).rvs((10, 100))
sim_prices = 151.03 * (1 + sim_returns).prod(axis=0)

# Calculate the current values and sim values
portfolios["CurrentValue"] = portfolios["CurrentPrice"] * portfolios["Holding"]
curr_values = portfolios.groupby('Portfolio')['CurrentValue'].sum()
sim_values = calculate_sim_values(portfolios, sim_prices, 10)

# Calculate the value difference
sim_value_changes = (sim_values.T - curr_values).T
curr_values

Portfolio
Call              6.80
CallSpread        4.59
CoveredCall     146.98
ProtectedPut    154.04
Put               4.85
PutSpread         3.01
Stock           151.03
Straddle         11.65
SynLong           1.95
Name: CurrentValue, dtype: float64

In [52]:
def cal_ES(x,alpha=0.05):
    xs = np.sort(x)
    n = alpha * len(xs)
    iup = math.ceil(n)
    idn = math.floor(n)
    VaR = (xs[iup] + xs[idn]) / 2
    ES = xs[0:idn].mean()
    return -VaR,-ES

# Calculate the Mean, VaR and ES, and print the results
resulting_mat = pd.DataFrame(0, index=sim_values.index.values, columns=["Mean", "VaR($)", "ES($)"])
for i in range(len(resulting_mat)):
    resulting_mat.iloc[i,0] = sim_values.iloc[i,:].mean()
    resulting_mat.iloc[i,1], resulting_mat.iloc[i,2] = cal_ES(sim_value_changes.iloc[i,:],alpha=0.05)[0:2]
resulting_mat

,Mean,VaR($),ES($)
Call,6.673385,6.193469,6.562662
CallSpread,4.064113,4.023378,4.362484
CoveredCall,145.246079,13.850938,18.803451
ProtectedPut,154.083535,7.643353,8.041389
Put,6.804384,4.352356,4.649988
PutSpread,3.828833,2.625317,2.854175
Stock,149.519227,17.739029,22.800091
Straddle,13.477769,1.346503,1.385781
SynLong,-0.130999,19.005682,24.336698


In [58]:
S = 151.03
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1

cal_amr_delta_num = cal_partial_derivative(binomial_tree, 1, 'S0')

# Calculate the deltas_normal for all portfolios
deltas = []
for i in range(len(portfolios.index)):
    if portfolios["Type"][i] == "Stock":
        deltas.append(1)
    else:
        option_type = portfolios["OptionType"][i]
        X = portfolios["Strike"][i]
        T = ((portfolios["ExpirationDate"][i] - current_date).days - 10) / 365
        div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
        delta = cal_amr_delta_num(option_type, S, X, T, div_time, div, implied_vol, r, N)
        deltas.append(delta)

# Store the deltas in portfolios
portfolios["deltas"] = deltas

alpha = 0.05
t = 10
result_dn = pd.DataFrame(index=sorted(portfolios['Portfolio'].unique()), columns=['Mean', 'VaR', 'ES'])
result_dn.index.name = 'Portfolio'
for pfl, df in portfolios.groupby('Portfolio'):
    gradient = S / df['CurrentValue'].sum() * (df['Holding'] * df['deltas']).sum()
    pfl_10d_std = abs(gradient) * std * np.sqrt(t)
    N = scipy.stats.norm(0, 1)
    present_value = df['CurrentValue'].sum()
    result_dn.loc[pfl]['Mean'] = 0
    result_dn.loc[pfl]['VaR'] = -present_value * N.ppf(alpha) * pfl_10d_std
    result_dn.loc[pfl]['ES'] = present_value * pfl_10d_std * N.pdf(N.ppf(alpha)) / alpha

result_dn

,Mean,VaR,ES
Portfolio,,,
Call,0,9.51986,11.938288
CallSpread,0,5.26384,6.601068
CoveredCall,0,10.901461,13.670872
ProtectedPut,0,12.141348,15.225741
Put,0,8.187534,10.267498
PutSpread,0,4.91698,6.166092
Stock,0,17.627131,22.105134
Straddle,0,1.332326,1.67079
SynLong,0,17.707393,22.205786
